In [2]:
import yaml
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.colors as mcolors

from io import BytesIO
from PIL import Image

In [3]:
with open("./data/pokemon-forms.yaml", 'r') as file:
    data = yaml.safe_load(file)
data_list = [value for value in data.values()]
df = pd.DataFrame(data_list)

In [4]:
###############
####LIBRARY####
###############
import requests

def get_pokemon_image(pokemon_id):
    url = f"https://img.pokemondb.net/artwork/large/{pokemon_id}.jpg"
    
    try:
        response = requests.get(url)
        response.raise_for_status() # checks for errors
        
        return response.content
    
    except requests.RequestException as e:
        print(f"Error downloading image: {e}")
        
#inutile pour l'instant
def get_min_max(stat):
    temp = 0
    id = 0
    i = 0
    for index, row in df.iterrows():
        if row['stats'][stat] > temp:
            temp = row['stats'][stat]
            id = i
        i += 1
    return id
get_min_max('spdef')

1058

In [7]:
#################
####VARIABLES####
#################

colors = {
    "normal": "#A8A77A",
    "fighting": "#C22E28",
    "flying": "#A98FF3",
    "poison": "#A33EA1",
    "ground": "#E2BF65",
    "rock": "#B6A136",
    "bug": "#A6B91A",
    "ghost": "#735797",
    "steel": "#B7B7CE",
    "fire": "#EE8130",
    "water": "#6390F0",
    "grass": "#7AC74C",
    "electric": "#F7D02C",
    "psychic": "#F95587",
    "ice": "#96D9D6",
    "dragon": "#6F35FC",
    "dark": "#705746",
    "fairy": "#D685AD"
}
stat_names = {
    'hp':'HP',
    'attack':'Attack',
    'defense':'Defense',
    'spatk':'Sp. Atk.',
    'spdef':'Sp. Def.',
    'speed':'Speed'
}

stats_statistics = pd.DataFrame(df['stats'])['stats'].apply(pd.Series).describe().round(2).to_dict()

stat_colormap = mcolors.LinearSegmentedColormap.from_list("pokemon_stat", ["red", "yellow", "lime"], N=160)

In [8]:
#########################
##POKEMON DISPLAY RADAR##
#########################

pokemon = df.iloc[2]
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=list(pokemon['stats'].values()),
    theta=list(pokemon['stats'].keys()),
    fill='toself',
    name=pokemon['pokemonid'],
    line=dict(color=colors[pokemon['type2']] if pokemon['type2'] else 'rgba(0,0,0,0.2)'),
    fillcolor=colors[pokemon['type1']],
    opacity=0.5
))

fig.add_annotation(
    x=0, y=0.8,
    text=f"Name: {pokemon['pokemonid'].capitalize()}",
    showarrow=False,
)
fig.add_annotation(
    x=0, y=0.7,
    text=f"Types: {pokemon['type1']} - {pokemon['type2']}",
    showarrow=False,
)

fig.add_layout_image(
        dict(
            source=Image.open(BytesIO(get_pokemon_image(pokemon['pokemonid']))),
            x=0, y=0.6,  
            sizex=0.5,
            sizey=0.5
        )
    )
#print(".")#don't show the plot for space

In [9]:
###########################
##POKEMON DISPLAY BARPLOT##
###########################

pokemon = df.iloc[182]
fig = go.Figure()

# barplot
for stat, value in pokemon['stats'].items():
    fig.add_trace(go.Bar(
        y=[f"{stat_names[stat]} "],
        x=[value],
        text=[f"<b>{value}</b>"],
        orientation='h',
        marker_color=mcolors.to_hex(stat_colormap(value)),
        showlegend=False,
        opacity=0.7
    ))

fig.update_layout(
    title=f"{pokemon['pokemonid'].capitalize()}",
    height=300,
    margin=dict(l=200),
    xaxis=dict(
        domain=[0.4, 1],
        range=[0, 255]
    )
)

#Image du pokémon
fig.add_layout_image(
        dict(
            source=Image.open(BytesIO(get_pokemon_image(pokemon['pokemonid']))),
            x=-0.2, y=1,  
            sizex=1,
            sizey=1
        )
    )

In [ ]:
#FAIRE UN RESEAU DIRECTIONNEL DES TYPES GENRE LIE QUI BAT QUI avec les faiblessent/résistance etc...

In [ ]:
# (faut bien qu'on rende un site avec dash, pas un notebook)